In [1]:
from itertools import permutations
import json
from matplotlib import pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw

%matplotlib inline

In [2]:
nitric_acid = "O[N+](=O)[O-]"
ammonia = "N"
aniline = "c1ccc(N)cc1"
hydrogen = "[H][H]"
hydrazine = "NN"
chlorine = "ClCl"
ethyl_alcohol = "CCO"
isopropyl_alcohol = "CC(O)C"
acetic_acid = "CC(=O)O"
phenol = "c1cc(O)ccc1"
diethyl_ether = "CCOCC"
dimethyl_ether = "COC"

In [3]:
ar_nu = "aromatic nucleophilic substitution"
ar_el = "aromatic electrophilic substitution"

In [4]:
ewgs = [
    "[N+](=O)[O-]", "C(=O)CC", "C(=O)NC", 
    "C(F)(F)F", "C(Cl)(Cl)(Cl)", "C#N", "[N+](C)(C)C",
    "OS(=O)=O", "OS(=O)(=O)C"
]

edgs = [
    "N", "N(C)C", "C(C)(C)C", "O", "NC(=O)CC",
    "S", "SC"
]

halogens = [
    "F", "Cl", "Br", "I"
]

el_ins = ["ClCl", "CC(=O)Cl", "BrBr", "OS(=O)(=O)O", "O[N+](=O)[O-]"]
el_acs = ["Cl", "C(=O)C", "Br", "S(=O)(=O)O", "[N+](=O)O"]

In [5]:
tests = []

In [6]:
# Benzene ring. Inactive.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    reagents = ["c1ccccc1", el_in]
    
    product = "c1{}ccccc1".format(el_acf)
    
    paths = []
    
    tests.append({
            "regents": reagents,
            "product": product,
            "paths": paths
        })

In [7]:
# EDG in ortho-para. Active.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, "", ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            reagents = ["c1c{d_0}cc{d_1}cc{d_2}1".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c{el}1c{d_0}cc{d_1}cc{d_2}1".format(**perm_dict)

            paths = [[ar_el]]

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [8]:
# EDG in meta- . Inactive

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            reagents = ["c1cc{d_0}cc{d_1}c1".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c{el}1cc{d_0}cc{d_1}c1".format(**perm_dict)

            paths = []

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [9]:
# EDG and EWG. Interferring

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([edgf, ewgf, ""])):
                reagents = ["c1c{}cc{}cc{}1".format(*perm), el_in]
                
                product = "c{}1c{}cc{}cc{}1".format(el_acf, *perm)
                
                paths = []
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [10]:
# EDG and EWG. Non-interferring

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([ewgf, el_ac, ""])):
                perm_dict = dict([(k, v) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                rev_perm_dict = dict([(v, k) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                
                k_el = rev_perm_dict[el_ac]
                perm_dict[k_el] = ""
                
                perm_dict["d"] = edgf
                
                reagents = ["c{d}1c{s_0}cc{s_1}cc{s_2}1".format(**perm_dict), el_in]
                
                perm_dict[k_el] = el_acf
                product = "c{d}1c{s_0}cc{s_1}cc{s_2}1".format(**perm_dict)
                
                paths = [[ar_el]]
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [11]:
# EDGs in both meta- and ortho-/para- positions. Active, in several positions.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg1 in edgs:
        edg1f = "({})".format(edg1)
        for edg2 in edgs:
            edg2f = "({})".format(edg2)
            
            for perm_meta in set(permutations([edg1f, ""])):
                for perm_ortho in set(permutations([edg2f, "", ""])):
                    perm = dict([(k, v) for k, v in zip(["d_m_{}".format(i) for i in range(len(perm_meta))], perm_meta)])
                    perm.update(dict([(k, v) for k, v in zip(["d_o_{}".format(i) for i in range(len(perm_ortho))], perm_ortho)]))
                    reagents = ["c1c{d_o_0}c{d_m_0}c{d_o_1}c{d_m_1}c{d_o_2}1".format(**perm), el_in]
                    
                    perm["el"] = el_acf
                    product = "c{el}1c{d_o_0}c{d_m_0}c{d_o_1}c{d_m_1}c{d_o_2}1".format(**perm)
                    
                    paths = []
                    
                    tests.append({
                        "regents": reagents,
                        "product": product,
                        "paths": paths
                    })

In [12]:
# Naphtalene ring. Inactive.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    for perm in set(permutations([el_acf] + [""] * 7)):
        reagents = ["c1cccc2ccccc12", el_in]

        product = "c{}1c{}c{}c{}c2c{}c{}c{}c{}c12".format(*perm)

        paths = []

        tests.append({
                "regents": reagents,
                "product": product,
                "paths": paths
            })

In [13]:
# EDG in ortho-para. Active.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, "", "", ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            
            reagents = ["c1c{d_0}cc{d_1}c2c{d_2}cc{d_3}cc12".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c{el}1c{d_0}cc{d_1}c2c{d_2}cc{d_3}cc12".format(**perm_dict)

            paths = [[ar_el]]

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [14]:
# EDG in ortho-para. Another symmetry. Active.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, "", "", ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            
            reagents = ["c{d_0}1cc{d_1}cc2cc{d_2}cc{d_3}c12".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c{d_0}1c{el}c{d_1}cc2cc{d_2}cc{d_3}c12".format(**perm_dict)

            paths = [[ar_el]]

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [15]:
# EDG in meta- . Inactive

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, "", ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            reagents = ["c1cc{d_0}cc2cc{d_1}cc{d_2}c12".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c{el}1cc{d_0}cc2cc{d_1}cc{d_2}c12".format(**perm_dict)

            paths = []

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [16]:
# EDG in meta- . Another symmetry. Inactive

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg in edgs:
        edgf = "({})".format(edg)
        for perm in set(permutations([edgf, "", ""])):
            perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
            reagents = ["c1ccc{d_0}c2c{d_1}cc{d_2}cc12".format(**perm_dict), el_in]

            perm_dict["el"] = el_acf
            product = "c1c{el}cc{d_0}c2c{d_1}cc{d_2}cc12".format(**perm_dict)

            paths = []

            tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [17]:
# EDG and EWG. Interferring

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([edgf, ewgf, "", ""])):
                perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
                reagents = ["c1c{d_0}cc{d_1}c2c{d_2}cc{d_3}cc12".format(**perm_dict), el_in]
                perm_dict["el"] = el_acf
                product = "c{el}1c{d_0}cc{d_1}c2c{d_2}cc{d_3}cc12".format(**perm_dict)
                
                paths = []
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [18]:
# EDG and EWG. Interferring. Another symmetry.

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([edgf, ewgf, "", ""])):
                perm_dict = dict([(k, v) for k, v in zip(["d_{}".format(i) for i in range(len(perm))], perm)])
                reagents = ["c{d_0}1cc{d_1}cc2cc{d_2}cc{d_3}c12".format(**perm_dict), el_in]
                perm_dict["el"] = el_acf
                product = "c{d_0}1c{el}c{d_1}cc2cc{d_2}cc{d_3}c12".format(**perm_dict)
                
                paths = []
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [19]:
# EDG and EWG. Non-interferring

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([ewgf, el_ac, "", ""])):
                perm_dict = dict([(k, v) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                rev_perm_dict = dict([(v, k) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                
                k_el = rev_perm_dict[el_ac]
                perm_dict[k_el] = ""
                
                perm_dict["d"] = edgf
                
                reagents = ["c{d}1c{s_0}cc{s_1}c2c{s_2}cc{s_3}cc12".format(**perm_dict), el_in]
                
                perm_dict[k_el] = el_acf
                product = "c{d}1c{s_0}cc{s_1}c2c{s_2}cc{s_3}cc12".format(**perm_dict)
                
                paths = [[ar_el]]
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [20]:
# EDG and EWG. Non-interferring. Another symmetry.

for edg in edgs:
    edgf = "({})".format(edg)
    for ewg in ewgs:
        ewgf = "({})".format(ewg)
        for el_in, el_ac in zip(el_ins, el_acs):
            el_acf = "({})".format(el_ac)
            for perm in set(permutations([ewgf, el_ac, "", ""])):
                perm_dict = dict([(k, v) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                rev_perm_dict = dict([(v, k) for k, v in zip(["s_%d" % i for i in range(len(perm))], perm)])
                
                k_el = rev_perm_dict[el_ac]
                perm_dict[k_el] = ""
                
                perm_dict["d"] = edgf
                
                reagents = ["c{s_0}1c{d}c{s_1}cc2cc{s_2}cc{s_3}c12".format(**perm_dict), el_in]
                
                perm_dict[k_el] = el_acf
                product = "c{s_0}1c{d}c{s_1}cc2cc{s_2}cc{s_3}c12".format(**perm_dict)
                
                paths = [[ar_el]]
                
                tests.append({
                    "regents": reagents,
                    "product": product,
                    "paths": paths
                })

In [21]:
# EDGs in both meta- and ortho-/para- positions. Active, in several positions.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg1 in edgs:
        edg1f = "({})".format(edg1)
        for edg2 in edgs:
            edg2f = "({})".format(edg2)
            
            for perm_meta in set(permutations([edg1f, "", ""])):
                for perm_ortho in set(permutations([edg2f, "", "", ""])):
                    perm = dict([(k, v) for k, v in zip(["d_m_{}".format(i) for i in range(len(perm_meta))], perm_meta)])
                    perm.update(dict([(k, v) for k, v in zip(["d_o_{}".format(i) for i in range(len(perm_ortho))], perm_ortho)]))
                    
                    reagents = ["c1c{d_o_0}c{d_m_0}c{d_o_1}c2c{d_o_2}c{d_m_1}c{d_o_3}c{d_m_2}c12".format(**perm), el_in]
                    
                    perm["el"] = el_acf
                    product = "c{el}1c{d_o_0}c{d_m_0}c{d_o_1}c2c{d_o_2}c{d_m_1}c{d_o_3}c{d_m_2}c12".format(**perm)
                    
                    paths = []
                    
                    tests.append({
                        "regents": reagents,
                        "product": product,
                        "paths": paths
                    })

In [22]:
# EDGs in both meta- and ortho-/para- positions. Active, in several positions. Another symmetry.

for el_in, el_ac in zip(el_ins, el_acs):
    el_acf = "({})".format(el_ac)
    
    for edg1 in edgs:
        edg1f = "({})".format(edg1)
        for edg2 in edgs:
            edg2f = "({})".format(edg2)
            
            for perm_meta in set(permutations([edg1f, "", ""])):
                for perm_ortho in set(permutations([edg2f, "", "", ""])):
                    perm = dict([(k, v) for k, v in zip(["d_m_{}".format(i) for i in range(len(perm_meta))], perm_meta)])
                    perm.update(dict([(k, v) for k, v in zip(["d_o_{}".format(i) for i in range(len(perm_ortho))], perm_ortho)]))
                    
                    reagents = ["c{d_o_0}1cc{d_o_1}c{d_m_0}c2c{d_m_1}c{d_o_2}c{d_m_2}c{d_o_3}c12".format(**perm), el_in]
                    
                    perm["el"] = el_acf
                    product = "c{d_o_0}1c{el}c{d_o_1}c{d_m_0}c2c{d_m_1}c{d_o_2}c{d_m_2}c{d_o_3}c12".format(**perm)
                    
                    paths = []
                    
                    tests.append({
                        "regents": reagents,
                        "product": product,
                        "paths": paths
                    })

In [24]:
json.dump(tests, open("../cases/electro_carbo_aro.json", "w"))